# Access locations and times of Veery encounters

For this challenge, you will use a database called the [Global
Biodiversity Information Facility (GBIF)](https://www.gbif.org/). GBIF
is compiled from species observation data all over the world, and
includes everything from museum specimens to photos taken by citizen
scientists in their backyards.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Explore GBIF</div></div><div class="callout-body-container callout-body"><p>Before your get started, go to the <a
href="https://www.gbif.org/occurrence/search">GBIF occurrences search
page</a> and explore the data.</p></div></div>

> **Contribute to open data**
>
> You can get your own observations added to GBIF using
> [iNaturalist](https://www.inaturalist.org/)!

### Set up your code to prepare for download

We will be getting data from a source called [GBIF (Global Biodiversity
Information Facility)](https://www.gbif.org/). We need a package called
`pygbif` to access the data, which may not be included in your
environment. Install it by running the cell below:

In [1]:
%%bash
pip install pygbif

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Import packages</div></div><div class="callout-body-container callout-body"><p>In the imports cell, we’ve included some packages that you will need.
Add imports for packages that will help you:</p>
<ol type="1">
<li>Work with reproducible file paths</li>
<li>Work with tabular data</li>
</ol></div></div>

In [1]:
import os
import pathlib
import time
import zipfile
import pandas as pd
from getpass import getpass
from glob import glob

import pygbif.occurrences as occ
import pygbif.species as species

In [6]:
# Create data directory in the home folder
data_dir = os.path.join(
    # Home directory
    pathlib.Path.home(),
    # Earth analytics data directory
    'earth-analytics',
    'data',
    # Project directory
    'spec_dist_pacific_loon',
)
os.makedirs(data_dir, exist_ok=True)

# Define the directory name for GBIF data
gbif_dir = os.path.join(data_dir, 'spec_dist_pacific_loon')

:::

### Register and log in to GBIF

You will need a [GBIF account](https://www.gbif.org/) to complete this
challenge. You can use your GitHub account to authenticate with GBIF.
Then, run the following code to save your credentials on your computer.

> **Warning**
>
> Your email address **must** match the email you used to sign up for
> GBIF!

> **Tip**
>
> If you accidentally enter your credentials wrong, you can set
> `reset_credentials=True` instead of `reset_credentials=False`.

In [21]:
reset_credentials = True
# GBIF needs a username, password, and email
credentials = dict(
    GBIF_USER=(input, ''),
    GBIF_PWD=(getpass, ''),
    GBIF_EMAIL=(input, ''),
)
for env_variable, (prompt_func, prompt_text) in credentials.items():
    # Delete credential from environment if requested
    if reset_credentials and (env_variable in os.environ):
        os.environ.pop(env_variable)
    # Ask for credential and save to environment
    if not env_variable in os.environ:
        os.environ[env_variable] = prompt_func(prompt_text)

### Get the species key

> ** Your task**
>
> 1.  Replace the `species_name` with the name of the species you want
>     to look up
> 2.  Run the code to get the species key

In [22]:
# Query species
species_info = species.name_lookup('Gavia pacifica', rank='SPECIES')
species_info

{'offset': 0,
 'limit': 100,
 'endOfRecords': True,
 'count': 32,
 'results': [{'key': 177664845,
   'datasetKey': '6b6b2923-0a10-4708-b170-5b7c611aceef',
   'nubKey': 2481955,
   'parentKey': 177664839,
   'parent': 'Gavia',
   'kingdom': 'Metazoa',
   'phylum': 'Chordata',
   'order': 'Gaviiformes',
   'family': 'Gaviidae',
   'genus': 'Gavia',
   'species': 'Gavia pacifica',
   'kingdomKey': 177651702,
   'phylumKey': 177654008,
   'classKey': 177656782,
   'orderKey': 177664834,
   'familyKey': 177664836,
   'genusKey': 177664839,
   'speciesKey': 177664845,
   'scientificName': 'Gavia pacifica',
   'canonicalName': 'Gavia pacifica',
   'nameType': 'SCIENTIFIC',
   'taxonomicStatus': 'ACCEPTED',
   'rank': 'SPECIES',
   'origin': 'SOURCE',
   'numDescendants': 0,
   'numOccurrences': 0,
   'habitats': [],
   'nomenclaturalStatus': [],
   'threatStatuses': [],
   'descriptions': [],
   'vernacularNames': [],
   'higherClassificationMap': {'177651702': 'Metazoa',
    '177654008': 'Ch

In [23]:
# Query species
species_info = species.name_lookup('Gavia pacifica', rank='SPECIES')

# Get the first result
first_result = species_info['results'][0]

# Get the species key (nubKey)
species_key = first_result['nubKey']

# Check the result
first_result['species'], species_key

('Gavia pacifica', 2481955)

### Download data from GBIF

::: {.callout-task title=“Submit a request to GBIF”

1.  Replace `csv_file_pattern` with a string that will match **any**
    `.csv` file when used in the `glob` function. HINT: the character
    `*` represents any number of any values except the file separator
    (e.g. `/`)

2.  Add parameters to the GBIF download function, `occ.download()` to
    limit your query to:

    -   observations
    -   from 2023
    -   with spatial coordinates.

3.  Then, run the download. **This can take a few minutes**. :::

In [30]:
# Only download once
gbif_pattern = os.path.join(gbif_dir, '*.csv')
if not glob(gbif_pattern):
    # Only submit one request
    if not 'GBIF_DOWNLOAD_KEY' in os.environ:
        # Submit query to GBIF
        gbif_query = occ.download([
        "speciesKey = 2481955",
        "year = 2023",
        "hasCoordinate = True"
        ])
        os.environ['GBIF_DOWNLOAD_KEY'] = gbif_query[0]

    # Wait for the download to build
    download_key = os.environ['GBIF_DOWNLOAD_KEY']
    wait = occ.download_meta(download_key)['status']
    while not wait=='SUCCEEDED':
        wait = occ.download_meta(download_key)['status']
        time.sleep(5)

    # Download GBIF data
    download_info = occ.download_get(
        os.environ['GBIF_DOWNLOAD_KEY'], 
        path=data_dir)

    # Unzip GBIF data
    with zipfile.ZipFile(download_info['path']) as download_zip:
        download_zip.extractall(path=gbif_dir)

# Find the extracted .csv file path (take the first result)
gbif_path = glob(gbif_pattern)[0]

INFO:Your download key is 0030807-241007104925546
INFO:Download file size: 2242035 bytes
INFO:On disk at /home/jovyan/earth-analytics/data/spec_dist_pacific_loon/0030807-241007104925546.zip


In [31]:
gbif_pattern = os.path.join(gbif_dir, '*.csv')
# Find the extracted .csv file path (take the first result)
gbif_path = glob(gbif_pattern)[0]
gbif_path

'/home/jovyan/earth-analytics/data/spec_dist_pacific_loon/spec_dist_pacific_loon/0030807-241007104925546.csv'

### Load the GBIF data into Python

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Load GBIF data</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Look at the beginning of the file you downloaded using the code
below. What do you think the <strong>delimiter</strong> is?</li>
<li>Run the following code cell. What happens?</li>
<li>Uncomment and modify the parameters of <code>pd.read_csv()</code>
below until your data loads successfully and you have only the columns
you want.</li>
</ol></div></div>

You can use the following code to look at the beginning of your file:

In [32]:
!head -n 2 $gbif_path 

gbifID	datasetKey	occurrenceID	kingdom	phylum	class	order	family	genus	species	infraspecificEpithet	taxonRank	scientificName	verbatimScientificName	verbatimScientificNameAuthorship	countryCode	locality	stateProvince	occurrenceStatus	individualCount	publishingOrgKey	decimalLatitude	decimalLongitude	coordinateUncertaintyInMeters	coordinatePrecision	elevation	elevationAccuracy	depth	depthAccuracy	eventDate	day	month	year	taxonKey	speciesKey	basisOfRecord	institutionCode	collectionCode	catalogNumber	recordNumber	identifiedBy	dateIdentified	license	rightsHolder	recordedBy	typeStatus	establishmentMeans	lastInterpreted	mediaType	issue
4953151418	50c9509d-22c7-4a22-a47d-8c48425ef4a7	https://www.inaturalist.org/observations/176167268	Animalia	Chordata	Aves	Gaviiformes	Gaviidae	Gavia	Gavia pacifica		SPECIES	Gavia pacifica (Lawrence, 1858)	Gavia pacifica		CA		Manitoba	PRESENT		28eb1a3f-1c15-4a95-931a-4af90ecb574d	58.765791	-94.122485							2023-08-01T14:50:18	1	8	2023	2481955	2481955	HUMAN_OBSERV

In [51]:
# Load the GBIF data
gaviapac_gbif_df = pd.read_csv(
    gbif_path,
    sep='\t',
    index_col='gbifID',
    header='infer',
    usecols=['gbifID', 'occurrenceID', 'species', 'scientificName', 
             'countryCode', 'occurrenceStatus', 'individualCount',
             'decimalLatitude', 'decimalLongitude', 'month', 'year', 
             'speciesKey', 'basisOfRecord']
    )
gaviapac_gbif_df.head()

,occurrenceID,species,scientificName,countryCode,occurrenceStatus,individualCount,decimalLatitude,decimalLongitude,month,year,speciesKey,basisOfRecord
gbifID,,,,,,,,,,,,
4953151418,https://www.inaturalist.org/observations/17616...,Gavia pacifica,"Gavia pacifica (Lawrence, 1858)",CA,PRESENT,NaN,58.765791,-94.122485,8,2023,2481955,HUMAN_OBSERVATION
4950273871,https://www.inaturalist.org/observations/24385...,Gavia pacifica,"Gavia pacifica (Lawrence, 1858)",US,PRESENT,NaN,32.758759,-117.245769,11,2023,2481955,HUMAN_OBSERVATION
4946632056,https://www.inaturalist.org/observations/23034...,Gavia pacifica,"Gavia pacifica (Lawrence, 1858)",US,PRESENT,NaN,36.909267,-122.026831,11,2023,2481955,HUMAN_OBSERVATION
4937187752,https://www.inaturalist.org/observations/14980...,Gavia pacifica,"Gavia pacifica (Lawrence, 1858)",US,PRESENT,NaN,49.000610,-123.167541,2,2023,2481955,HUMAN_OBSERVATION
4936192321,https://www.inaturalist.org/observations/19074...,Gavia pacifica,"Gavia pacifica (Lawrence, 1858)",US,PRESENT,NaN,34.725271,-118.167093,11,2023,2481955,HUMAN_OBSERVATION


In [52]:
basisOfRecord_table = gaviapac_gbif_df['basisOfRecord'].value_counts() 
print(basisOfRecord_table) 

basisOfRecord
HUMAN_OBSERVATION    33715
Name: count, dtype: int64


In [53]:
%store gaviapac_gbif_df

Stored 'gaviapac_gbif_df' (DataFrame)
